In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import plotly.express as px

In [54]:
# Importando os dados
df_lag_cp = pd.read_csv('data/bem_comportadas_laguardia.csv')
df_lag_li = pd.read_csv('data/livres_laguardia.csv')
df_van_cp = pd.read_csv('data/bem_comportadas_vanessa.csv')
df_van_li = pd.read_csv('data/livres_vanessa.csv')
df_bru_cp = pd.read_csv('data/bem_comportadas_bruno.csv')
df_bru_li = pd.read_csv('data/livres_bruno.csv')

X_lag_cp = df_lag_cp.drop('pose', axis=1)
Y_lag_cp = df_lag_cp['pose'].astype('category')
X_lag_li = df_lag_li.drop('pose', axis=1)
Y_lag_li = df_lag_li['pose'].astype('category')
X_van_cp = df_van_cp.drop('pose', axis=1)
Y_van_cp = df_van_cp['pose'].astype('category')
X_van_li = df_van_li.drop('pose', axis=1)
Y_van_li = df_van_li['pose'].astype('category')
X_bru_cp = df_bru_cp.drop('pose', axis=1)
Y_bru_cp = df_bru_cp['pose'].astype('category')
X_bru_li = df_bru_li.drop('pose', axis=1)
Y_bru_li = df_bru_li['pose'].astype('category')

# Juntando os dados de treino e teste
X_train = pd.concat([X_lag_cp, X_van_cp, X_bru_cp])
Y_train = pd.concat([Y_lag_cp, Y_van_cp, Y_bru_cp])
X_test = pd.concat([X_lag_li, X_van_li, X_bru_li])
Y_test = pd.concat([Y_lag_li, Y_van_li, Y_bru_li])

# Passando média móvel nos dados de pressão
for col_name in X_train.columns:
    if col_name[0] == 'p':
        X_train[col_name] = X_train[col_name].rolling(10).mean()
        X_test[col_name] = X_test[col_name].rolling(10).mean()

# Removendo as 10 primeiras linhas - NaNs da média móvel
X_train = X_train.iloc[10:]
Y_train = Y_train.iloc[10:]
X_test = X_test.iloc[10:]
Y_test = Y_test.iloc[10:]

In [3]:
# ===== Modelo ===== #
def instantiate_model(seed, num_neurons, num_hidden_layers):
    tf.random.set_seed(seed)
    
    model_layers = []
    model_layers.append(tf.keras.layers.Dense(2**num_neurons, activation='relu', input_shape=(X_lag_cp.shape[1],)))
    for i in range(num_hidden_layers):
        model_layers.append(tf.keras.layers.Dense(2**num_neurons, activation='relu'))
    model_layers.append(tf.keras.layers.Dense(13, activation='softmax'))

    model = tf.keras.models.Sequential(model_layers)

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    return model

In [57]:
model = instantiate_model(1, 7, 2)

# ===== Treinamento ===== #
EPOCHS = 20
loss_train = []
loss_test = []
for epoch in range(EPOCHS):
    print(f'Epoch {epoch+1}/{EPOCHS}', end='\r')
    model.fit(X_train, Y_train, epochs=1, verbose=0)
    loss_train.append(model.evaluate(X_train, Y_train, verbose=0)[0])
    loss_test.append(model.evaluate(X_test, Y_test, verbose=0)[0])

In [58]:
data = pd.DataFrame({'epoch': range(EPOCHS), 'loss_train': loss_train, 'loss_test': loss_test})

fig = px.line(data, x='epoch', y=['loss_train', 'loss_test'], title='Loss')
fig.show()